In [1]:
from cblearn import datasets
import numpy as np
import cblearn
import tqdm
import time
import sys
sys.path.append('../..')
from comparisonHC import HandlerTriplets, ListTriplets, get_AddS_triplets, get_MulK_triplets, get_tSTE_triplets, ComparisonHC

In [2]:
#fetching food dataset
data = datasets.fetch_food_similarity()

In [3]:
data

{'data': array([[88,  9, 51],
        [62, 48,  7],
        [95, 28,  6],
        ...,
        [ 1, 92, 97],
        [82, 59, 73],
        [ 7, 33, 11]], dtype=int64),
 'image_names': array(['images/00ca9e5230c749439a4b320e696e3819.jpg',
        'images/0180803ec54c4e0ab8dc8f9f5d41cc13.jpg',
        'images/028869b776a14002b6b63c4387f59815.jpg',
        'images/052f17a242a04ca288501afdbd81284f.jpg',
        'images/05681f3d14244879b2e076144440fda1.jpg',
        'images/08afe513729b41a59731b39a8a001c57.jpg',
        'images/0b9289f4b4c64d03a547a9ae6f5c026d.jpg',
        'images/0e37c510bb504d21baf495365c71ee16.jpg',
        'images/12f6787b3c24466bb149604f1ca00866.jpg',
        'images/13e60bef87794072bb613abcd61ba94d.jpg',
        'images/167766958abf4eb0b714dac6c16ae063.jpg',
        'images/1bb27004724346daa4878551c82e7b23.jpg',
        'images/20063713e2984fbab8d7efced8e40f7c.jpg',
        'images/214649bfd7ea489b8daf588e6fed45aa.jpg',
        'images/215931a9c4a54b04bdcdf8dfb7b9423

In [4]:
#getting raw triplets from car data
triplets_raw = data['data']

In [5]:
n = np.amax(triplets_raw) + 1

In [6]:
#objects in the data
n

100

In [7]:
n_triplets_raw = triplets_raw.shape[0]

In [8]:
#raw triplets in the data(number of responses)
n_triplets_raw

190376

In [9]:
#getting standard triplets from raw triplets. Random triplets are of form (i,j,k) where s_ij>s_ik
#raw triplets were already in standard form
triplets = triplets_raw

In [10]:
n_triplets = triplets.shape[0]

In [11]:
#standard triplets obtained from the data
n_triplets

190376

In [12]:
Oracle = ListTriplets(triplets, n, n_triplets)

In [13]:
adds_similarities = get_AddS_triplets(Oracle,n)

In [14]:
#running comparison hierarchical clustering using AddS-3 similarities
chc = ComparisonHC(adds_similarities,n)
chc.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc.time_elapsed))
print("Revenue with AddS-3: {}".format(-chc.cost_dasgupta(adds_similarities)))

ComparisonHC ran for 3.20 seconds.
Revenue with AddS-3: 6137273.0


In [15]:
mulk_similarities = get_MulK_triplets(Oracle,n)

In [16]:
#running comparison hierarchical clustering using MulK-3 similarities
chc_mulk = ComparisonHC(mulk_similarities,n)
chc_mulk.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc_mulk.time_elapsed))
print("Revenue with Mulk-3: {}".format(-chc_mulk.cost_dasgupta(adds_similarities)))

ComparisonHC ran for 9.56 seconds.
Revenue with Mulk-3: 6096110.0


In [17]:
#running comparison hierarchical clustering using tSTE similarities
#since it is a randomised process it is run 10 times and mean and standard deviation are obtained
tste_cost = []
total_time = 0.0
for i in tqdm.tqdm(range(10)):
    tste_similarities = get_tSTE_triplets(Oracle,n)
    chc_tste = ComparisonHC(tste_similarities,n)
    chc_tste.fit([[j] for j in range(n)])
    total_time = total_time+chc_tste.time_elapsed
    tste_cost.append(-chc_tste.cost_dasgupta(adds_similarities))
print("ComparisonHC ran for {:.2f} seconds.".format(total_time))

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [30:57<00:00, 185.73s/it]

ComparisonHC ran for 67.86 seconds.


In [18]:
tste_cost = np.array(tste_cost)
mean = np.mean(tste_cost)
std = np.std(tste_cost)

In [19]:
print("Mean revenue obtained with tSTE after 10 runs: {}".format(mean))

Mean revenue obtained with tSTE after 10 runs: 5999456.6


In [20]:
print("Standard deviation of revenue obtained with tSTE after 10 runs: {}".format(std))

Standard deviation of revenue obtained with tSTE after 10 runs: 6749.2174835309615
